In [96]:
import pandas as pd

# Anzeigeoptionen anpassen
pd.set_option('display.max_rows', None)    # zeigt alle Zeilen
pd.set_option('display.max_columns', None) # zeigt alle Spalten
pd.set_option('display.width', None)       # keine Begrenzung der Zeilenbreite
pd.set_option('display.max_colwidth', None)  # keine Begrenzung der Spaltenbreite


In [97]:
df_extended = pd.read_csv('Data/Speed_Dating_Data.csv', encoding='latin1')
df = pd.read_csv('data/flat-ui__data-Thu Jun 19 2025.csv', encoding='latin1')

In [98]:
df_extended.describe(include='all')

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,8378.000000,8368.000000,8378.000000,8220.000000,8378.000000,8274.000000,8305.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,8378.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,8283.000000,8315,8296.000000,4914,3133,3583,8315.000000,8299.000000,8299.000000,8299,7314,4279,8299.000000,8281.000000,8299.000000,8289,8240.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8277.000000,1800.000000,8299.000000,8299.000000,8299.000000,8289.000000,8279.000000,8257.000000,6489.000000,6489.000000,6489.000000,6489.000000,6489.000000,6467.000000,8299.000000,8299.000000,8299.000000,8299.000000,8289.000000,8289.000000,8273.000000,8273.000000,8273.000000,8273.000000,8273.000000,4906.000000,4906.000000,4906.000000,4906.000000,4906.000000,8378.000000,8176.000000,8101.000000,8082.000000,8028.000000,7666.000000,7311.000000,8138.000000,8069.000000,8003.000000,7205.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,7463.000000,7463.000000,7433.000000,1984.000000,1955.000000,1984.000000,1984.000000,1955.000000,1974.000000,7445.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.00000,5775.000000,5775.000000,5775.000000,5775.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,4377.000000,4377.000000,4377.000000,4377.000000,4377.000000,3974.000000,3974.000000,3974.000000,1496.000000,668.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2016.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259,NaN,241,68,115,NaN,NaN,NaN,269,409,261,NaN,NaN,NaN,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

Das ist um nacher noch die Wave 6-9 zu fixen

In [99]:
import numpy as np

# Variante A: mit einem kleinen Helfer, der in-place modifiziert und das DF zurückgibt
def multiply_wave_cols(df):
    # Maske für Zeilen 6–9
    mask = df['wave'].between(6, 9)
    # Spaltenauswahl: alle, die auf '_1' enden
    cols = ["attr4_1", "sinc4_1", "intel4_1", "fun4_1", "amb4_1", "shar4_1", "attr4_3", "sinc4_3", "intel4_3", "fun4_3", "amb4_3", "shar4_3",
            "attr2_3", "sinc2_3", "intel2_3", "fun2_3", "amb2_3", "shar2_3"]
    # in-place Multiplikation
    df.loc[mask, cols] *= 10
    return df


## Fitting Test XGB RF LM

In [100]:
df_extended = multiply_wave_cols(df_extended)

In [101]:
from sklearn.model_selection import train_test_split

# Spalten entfernen da sie 'Match' determinieren
df_extended = df_extended.drop(columns=['iid', 'id', 'idg', 'pid', "dec", "dec_o"], errors='ignore')

# Entferne jede Spalte die mehr als 5% fehlende werte hat 
threshold = 0.05 * len(df_extended)
cols_to_drop = df_extended.columns[df_extended.isnull().sum() >= threshold]
print("Spalten, die gelöscht werden (mehr als 5% fehlende Werte):")
print(cols_to_drop.tolist())
df_extended = df_extended.loc[:, df_extended.isnull().sum() < threshold]

# Train Test Split
X = df_extended.drop(["match"], axis=1)
y = df_extended["match"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Spalten, die gelöscht werden (mehr als 5% fehlende Werte):
['positin1', 'amb_o', 'shar_o', 'undergra', 'mn_sat', 'tuition', 'zipcode', 'income', 'expnum', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'amb', 'shar', 'match_es', 'attr1_s', 'sinc1_s', 'intel1_s', 'fun1_s', 'amb1_s', 'shar1_s', 'attr3_s', 'sinc3_s', 'intel3_s', 'fun3_s', 'amb3_s', 'satis_2', 'length', 'numdat_2', 'attr7_2', 'sinc7_2', 'intel7_2', 'fun7_2', 'amb7_2', 'shar7_2', 'attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2', 'attr4_2', 'sinc4_2', 'intel4_2', 'fun4_2', 'amb4_2', 'shar4_2', 'attr2_2', 'sinc2_2', 'intel2_2', 'fun2_2', 'amb2_2', 'shar2_2', 'attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2', 'attr5_2', 'sinc5_2', 'intel5_2', 'fun5_2', 'amb5_2', 'you_call', 'them_cal', 'date_3', 'numdat_3', 'num_in_3', 'attr1_3', 'sinc1_3', 'intel1_3', 'fun1_3', 'amb1_3', 'shar1_3', 'attr7_3', 'sinc7_3', 'intel7_3', 'fun7_3', 'amb7_3', 's

In [102]:
# Zeig mir alle nunique Werte von den Object spalten
object_columns = df_extended.select_dtypes(include=['object']).columns
for col in object_columns:
    print(f"Spalte: {col}, Anzahl der eindeutigen Werte: {df_extended[col].nunique()}")
    print("\n")

Spalte: field, Anzahl der eindeutigen Werte: 259


Spalte: from, Anzahl der eindeutigen Werte: 269


Spalte: career, Anzahl der eindeutigen Werte: 367




- pf_o: Preference of Partner 
- attr1_1: Preference Subject
- 
- attr_o: Rating of Partner 
- attr: Rating Partner
- 
- attr2_1: Perceived Preference Partner 
- attr3_1: Perceived Ratings



In [103]:
X.head()

,gender,condtn,wave,round,position,order,partner,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,intel_o,fun_o,like_o,prob_o,met_o,age,field,field_cd,race,imprace,imprelig,from,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr,sinc,intel,fun,like,prob,met
0,0,1,1,10,7,4,1,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,7.0,4.0,2.0,21.0,Law,1.0,4.0,2.0,4.0,Chicago,2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,6.0,9.0,7.0,7.0,7.0,6.0,2.0
1,0,1,1,10,7,3,2,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,8.0,4.0,2.0,21.0,Law,1.0,4.0,2.0,4.0,Chicago,2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,7.0,8.0,7.0,8.0,7.0,5.0,1.0
2,0,1,1,10,7,10,3,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,4.0,2.0,4.0,Chicago,2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,5.0,8.0,9.0,8.0,7.0,NaN,1.0
3,0,1,1,10,7,5,4,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,7.0,7.0,2.0,21.0,Law,1.0,4.0,2.0,4.0,Chicago,2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,7.0,6.0,8.0,7.0,7.0,6.0,2.0
4,0,1,1,10,7,7,5,0.21,0,24.0,3.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,8.0,6.0,2.0,21.0,Law,1.0,4.0,2.0,4.0,Chicago,2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,5.0,6.0,7.0,7.0,6.0,6.0,2.0


In [104]:
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline       import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import CountEncoder
from sklearn.svm import SVC
from xgboost                import XGBClassifier
from sklearn.linear_model   import LogisticRegression
from sklearn.ensemble       import RandomForestClassifier
import joblib
from sklearn.metrics import (
    accuracy_score, confusion_matrix, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report
)



#Funktion zum Erzeugen des passenden Preprocessors
def make_preprocessor(model_name, numeric_features, categorical_features):
    cat_encoder = encoding_strategies[model_name]
    categorical_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", cat_encoder)
    ])
    return ColumnTransformer([
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_pipeline, categorical_features)
    ])

# Numerische und kategoriale Features ermitteln (unverändert)
numeric_features = X.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

# Transformer für numerische und kategoriale Daten definieren (unverändert)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),      # fehlende Werte durch Median ersetzen
    ("scaler", StandardScaler())                        # Standardisierung
])

# 2) Encoding-Strategien je Modell festlegen
encoding_strategies = {
    "LogisticRegression":TargetEncoder(handle_unknown="ignore"),
    "RandomForest":      OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    "XGBoost":           OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    "KNN":               CountEncoder(handle_unknown="ignore"),
    "SVM":               TargetEncoder(handle_unknown="ignore")
}

models = [
    ("LogisticRegression", LogisticRegression(random_state=42)),
    ("RandomForest",       RandomForestClassifier(max_features= 3,random_state=42)),
    ("XGBoost",            XGBClassifier(random_state=42)),
    ("KNN",                KNeighborsClassifier()),
    ("SVM",                SVC(probability=True, random_state=42))
]

scoring = {
    "Accuracy":       "accuracy",
    "ROC_AUC":        "roc_auc_ovo_weighted",
    "F1":             "f1_macro"
}

for name, clf in models:
    pipe = Pipeline([
        ("preprocessor", make_preprocessor(name, numeric_features, categorical_features)),
        ("classifier",   clf)
    ])
    pipe.fit(X_train, y_train)

    # 2) Vorhersagen
    y_pred  = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] 
    
    # 3) Metriken berechnen
    acc   = accuracy_score(y_test, y_pred)
    f1    = f1_score(y_test, y_pred)
    roc   = roc_auc_score(y_test, y_proba)      # für Binary nur y_proba[:,1]
    
    # 4) Ausgabe
    print(f"\n=== {name} ===")
    print(f"Accuracy   : {acc:.3f}")
    print(f"F1-Score   : {f1:.3f}")
    print(f"ROC AUC    : {roc:.3f}")

    
    # 5) Modell speichern
    filename = f"model_{name}.pkl"
    joblib.dump(pipe, filename)


=== LogisticRegression ===
Accuracy   : 0.847
F1-Score   : 0.404
ROC AUC    : 0.846

=== RandomForest ===
Accuracy   : 0.848
F1-Score   : 0.270
ROC AUC    : 0.846

=== XGBoost ===
Accuracy   : 0.860
F1-Score   : 0.508
ROC AUC    : 0.865

=== KNN ===
Accuracy   : 0.828
F1-Score   : 0.300
ROC AUC    : 0.683

=== SVM ===
Accuracy   : 0.851
F1-Score   : 0.384
ROC AUC    : 0.858


In [76]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


CV für Hyperparameter und robuste unbiased Schätzung

In [ ]:
    cv_results = cross_validate(
        pipe,
        X_train, y_train,
        cv=5,
        scoring=scoring,
        return_train_score=False
    )
    mean_scores = {}
    for metric in scoring:
        m = cv_results[f"test_{metric}"]
        mean_scores[metric] = m.mean()
        print(f"  {metric:15s}: {m.mean():.3f} ± {m.std():.3f}")

    overall_avg = np.mean(list(mean_scores.values()))
    print(f"  {'Overall average':15s}: {overall_avg:.3f}")

## RF Model/Feature Importance

Sparse Group Lasso